In [5]:
#%%
import pandas as pd
import numpy as np
import joblib
from pyLDAvis import display
import matplotlib.pyplot as plt
pd.set_option('display.max_columns',None)
from collections import Counter

In [41]:
test_df = joblib.load('process_test.pkl')
joongratio = joblib.load('midcat_ratio.pkl')
goal_df = joblib.load('0216_0357_skill2pgm_formodel_ver1.pkl')

In [42]:
test_df

,showhost_input,weather_input,weekday_input,brand_input,midcat_input,price_input,expression_input,year_input,month_input,time_input,duration_input,live_input,holiday_input
0,"[왕영은, 하은지]",흐림,수,"[에이앤디, LG(엘지), 자연식품]","[여성캐쥬얼, 생활가전, 가공식품]","[55, 1697, 33]","[기간한정, 마지막(생방송)]",2022,3,19,120,True,False
1,"[유지수, 신진아]",맑음,토,"[용평리조트, 에이앤디, 헬스앤플러스, 삼성(SAMSUNG)]","[회원권/이용권, 여성정장, 헬스용품, 영상/음향]","[200, 50, 80, 300]",[방송에서만],2022,4,20,60,True,True
2,[최현우],맑음,토,[구찌],[여성(패션)],[2000],[없음],2022,4,17,178,True,True


In [53]:
# 전처리 함수 만들기 (main)

def process_input(df):
  # 자료형 변환
  int_cols = [
    'year_input',
    'month_input',
    'time_input',
    'duration_input',
  ]
  df[int_cols] = df[int_cols].astype(int)

  # col name change
  df = df.rename(columns={
    'year_input':'year',
    'month_input':'month',
    'showhost_input': '쇼호스트명',
    'weather_input': '날씨',
    'weekday_input': 'weekday',
    'duration_input': 'duration',
    'live_input': '상품구분',
    'holiday_input':'주말/공휴일'
  })

  # 상품명 대체 key 생성
  df['prd_key'] = df['midcat_input'].apply(lambda x: list(range(len(x))))
  #상품개수
  df['prod_num'] = df['price_input'].apply(lambda x: len(x))

  # data check
  df['prd_info_check'] = df.apply(
    lambda row: len(row['midcat_input']) != row['prod_num'] | len(row['brand_input']) != row['prod_num']
  ,axis=1)
  error_rows = df[df['prd_info_check']]
  if len(error_rows)!=0:
    raise Exception('상품중분류 - 상품 브랜드 - 상품 가격 입력 수가 같지 않습니다.')

  # 쇼호스트명 sorting
  df['쇼호스트명'] = df['쇼호스트명'].apply(lambda x: sorted(x))

  # 상품구분 변환
  df['상품구분'] = df['상품구분'].apply(lambda x: 'LIVE' if x else 'DATA')
  # 주말/공휴일 변환
  df['주말/공휴일'] = df['주말/공휴일'].apply(lambda x: 1 if x else 0)

  #year, month, hour
  df['year'] = df['year']-2020
  df['month' + '_sin'] = np.sin(2 * np.pi * df['month']/12)
  df['month' + '_cos'] = np.cos(2 * np.pi * df['month']/12)
  df['hour' + '_sin'] = np.sin(2 * np.pi * df['time_input']/24)
  df['hour' + '_cos'] = np.cos(2 * np.pi * df['time_input']/24)

  #판매가
  df['판매가평균'] = df['price_input'].apply(lambda x: np.mean(x)*1000)
  df['price_max_all'] = df['price_input'].apply(lambda x: max(x)*1000)
  df['price_min_all'] = df['price_input'].apply(lambda x: min(x)*1000)
  
  #한정표현 num
  df['한정표현_num'] = df['expression_input'].apply(lambda x: len(x))

  # sh_num
  df['sh_num'] = df['쇼호스트명'].apply(lambda x: len(x))

  return df
  

In [54]:
t = test_df.copy()
out = process_input(t)

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [47]:
out

,쇼호스트명,날씨,weekday,brand_input,midcat_input,price_input,expression_input,year,month,time_input,duration,상품구분,주말/공휴일,month_sin,month_cos,hour_sin,hour_cos,prod_num,판매가평균,price_max_all,price_min_all,한정표현_num,sh_num,prd_key
0,"[왕영은, 하은지]",흐림,수,"[에이앤디, LG(엘지), 자연식품]","[여성캐쥬얼, 생활가전, 가공식품]","[55, 1697, 33]","[기간한정, 마지막(생방송)]",2,3,19,120,LIVE,0,1.000000,6.123234e-17,-0.965926,0.258819,3,595000.0,1697000,33000,2,2,"[0, 1, 2]"
1,"[신진아, 유지수]",맑음,토,"[용평리조트, 에이앤디, 헬스앤플러스, 삼성(SAMSUNG)]","[회원권/이용권, 여성정장, 헬스용품, 영상/음향]","[200, 50, 80, 300]",[방송에서만],2,4,20,60,LIVE,1,0.866025,-5.000000e-01,-0.866025,0.500000,4,157500.0,300000,50000,1,2,"[0, 1, 2, 3]"
2,[최현우],맑음,토,[구찌],[여성(패션)],[2000],[없음],2,4,17,178,LIVE,1,0.866025,-5.000000e-01,-0.965926,-0.258819,1,2000000.0,2000000,2000000,1,1,[0]


In [48]:
# 상품명을 통해서 브랜드명, 가격을 함께 뽑아보아요ㅎㅎ
# 상품명을 key로, 중분류score를 value로 갖는 딕셔너리를 컬럼으로 넣어줍니다.
def makejoong(row):
    valueslist = row['midcat_input']
    prdslist = row['prd_key']
    scoredict = {}
    for i, value in enumerate(valueslist):
        scoredict[prdslist[i]] = joongratio[value]
    return scoredict

# score가 가장 높은 상품의 브랜드를 찾아주는 함수!
def makebrandcol(row): # 가장 큰 값으로 채우기
    brands = row['brand_input']
    prds = row['prd_key']
    scoredict = row['joongscore']
    collist = [] # 컬럼의 내용이 되는 리스트
    if len(scoredict) >= 3: # 상품 개수가 3개 이상
        for prdset in Counter(scoredict).most_common(3): # 각각의 상품에 대해서
            prdnm = prdset[0] # prdnm = 상품명
            index = prds.index(prdnm) # 상품명의 순서 추출
            collist.append(brands[index]) # 해당 순서의 브랜드 리스트에 넣어줍니다
    else: # 상품 개수가 3개 미만
        for prdset in Counter(scoredict).most_common(1):
            prdnm = prdset[0]
            index = prds.index(prdnm)
            collist.append(brands[index])
            collist.append(brands[index])
            collist.append(brands[index])
    return collist

# score가 가장 높은 상품의 가격을 찾아주는 함수!
def makepricecol(row): # 가장 큰 값으로 채우기
    prices = row['price_input']
    prds = row['prd_key']
    scoredict = row['joongscore']
    collist = [] # 컬럼의 내용이 되는 리스트
    if len(scoredict) >= 3: # 상품 개수가 3개 이상
        for prdset in Counter(scoredict).most_common(3):
            prdnm = prdset[0]
            index = prds.index(prdnm)
            collist.append(prices[index])
    else: # 상품 개수가 3개 미만
        for prdset in Counter(scoredict).most_common(1):
            prdnm = prdset[0]
            index = prds.index(prdnm)
            collist.append(prices[index])
            collist.append(prices[index])
            collist.append(prices[index])
    return collist

# score가 가장 높은 상품의 score만 남기는 함수!
def makescorecol(row): # 가장 큰 값으로 채우기
    scoredict = row['joongscore']
    collist = [] # 컬럼의 내용이 되는 리스트
    if len(scoredict) >= 3: # 상품 개수가 3개 이상
        for prdset in Counter(scoredict).most_common(3): # 각각의 상품에 대해서
            prdnm = prdset[0] # prdnm = 상품명
            collist.append(scoredict[prdnm])
    else: # 상품 개수가 3개 미만
        for prdset in Counter(scoredict).most_common(1):
            prdnm = prdset[0]
            collist.append(scoredict[prdnm])
            collist.append(scoredict[prdnm])
            collist.append(scoredict[prdnm])
    return collist


In [49]:
out['joongscore'] = out.apply(lambda row : makejoong(row), axis=1)
out['brands'] = out.apply(lambda row : makebrandcol(row), axis=1)
out['prices'] = out.apply(lambda row : makepricecol(row), axis=1)
out['scores'] = out.apply(lambda row : makescorecol(row), axis=1)

In [50]:
out.head()

,쇼호스트명,날씨,weekday,brand_input,midcat_input,price_input,expression_input,year,month,time_input,duration,상품구분,주말/공휴일,month_sin,month_cos,hour_sin,hour_cos,prod_num,판매가평균,price_max_all,price_min_all,한정표현_num,sh_num,prd_key,joongscore,brands,prices,scores
0,"[왕영은, 하은지]",흐림,수,"[에이앤디, LG(엘지), 자연식품]","[여성캐쥬얼, 생활가전, 가공식품]","[55, 1697, 33]","[기간한정, 마지막(생방송)]",2,3,19,120,LIVE,0,1.000000,6.123234e-17,-0.965926,0.258819,3,595000.0,1697000,33000,2,2,"[0, 1, 2]","{0: 0.1321001800622033, 1: 0.00226586102719033...","[에이앤디, 자연식품, LG(엘지)]","[55, 33, 1697]","[0.1321001800622033, 0.1141732283464567, 0.002..."
1,"[신진아, 유지수]",맑음,토,"[용평리조트, 에이앤디, 헬스앤플러스, 삼성(SAMSUNG)]","[회원권/이용권, 여성정장, 헬스용품, 영상/음향]","[200, 50, 80, 300]",[방송에서만],2,4,20,60,LIVE,1,0.866025,-5.000000e-01,-0.866025,0.500000,4,157500.0,300000,50000,1,2,"[0, 1, 2, 3]","{0: 0.004925209777453484, 1: 0.190277777777777...","[에이앤디, 헬스앤플러스, 용평리조트]","[50, 80, 200]","[0.19027777777777777, 0.007194244604316547, 0...."
2,[최현우],맑음,토,[구찌],[여성(패션)],[2000],[없음],2,4,17,178,LIVE,1,0.866025,-5.000000e-01,-0.965926,-0.258819,1,2000000.0,2000000,2000000,1,1,[0],{0: 0.05238095238095238},"[구찌, 구찌, 구찌]","[2000, 2000, 2000]","[0.05238095238095238, 0.05238095238095238, 0.0..."


In [ ]:
goal_df.head(1)

,인입콜,year,쇼호스트명,상품구분,날씨,weekday,duration,판매가평균,price_max_all,price_min_all,prod_num,한정표현_num,주말/공휴일,sh_num,month_sin,month_cos,hour_sin,hour_cos,시차변수,week,price_max_top3,price_min_top3,num1,num2,num3,brand1,brand2,brand3,price1,price2,price3,score1,score2,score3
0,33,0.0,"[엄지혜, 이경은]",DATA,구름 많음,수,40,99000.0,99000,99000,1,0,0,2.0,0.5,0.866025,1.0,6.123234e-17,NaN,1,99000,99000,3,3,3,소담,소담,소담,99000,99000,99000,0.045988,0.045988,0.045988


In [ ]:
goal_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34051 entries, 0 to 34050
Data columns (total 34 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   인입콜             34051 non-null  int64  
 1   year            34051 non-null  float64
 2   쇼호스트명           30949 non-null  object 
 3   상품구분            34051 non-null  object 
 4   날씨              34051 non-null  object 
 5   weekday         34051 non-null  object 
 6   duration        34051 non-null  int64  
 7   판매가평균           34051 non-null  float64
 8   price_max_all   34051 non-null  int64  
 9   price_min_all   34051 non-null  int64  
 10  prod_num        34051 non-null  int64  
 11  한정표현_num        34051 non-null  int64  
 12  주말/공휴일          34051 non-null  int64  
 13  sh_num          30949 non-null  float64
 14  month_sin       34051 non-null  float64
 15  month_cos       34051 non-null  float64
 16  hour_sin        34051 non-null  float64
 17  hour_cos        34051 non-null 